<span style="float: left;padding: 1.3em">![logo](https://github.com/jcvl-udg/odw-2024/blob/main/Tutorials/logo.png?raw=1)</span>

#  Gravitational Wave Open Data Workshop #7


#### Tutorial 1.1: Discovering open data from GW observatories

This notebook describes how to discover what data are available from the [Gravitational-Wave Open Science Center (GWOSC)](https://www.gw-openscience.org).
    
[Click this link to view this tutorial in Google Colaboratory](https://colab.research.google.com/github/gw-odw/odw-2024/blob/main/Tutorials/Day_1/Tuto_1.1_Discovering_Open_Data.ipynb)

## Software installation  (execute only if running on a cloud platform or haven't done the installation yet!)

First, we need to install the software, which we do following the instruction in [Software Setup Instructions](../../setup.md):

date: 08/07/2025
- La instalacion de la libreria en esa version causa conflictos, se instalara la mas actual por defecto(0.8.1)

In [4]:
# -- Uncomment following line if running in Google Colab
#! pip install -q gwosc

In [5]:
#check the version of the package gwosc you are using
import gwosc
print(gwosc.__version__)

0.8.1


*The version you get should be 0.7.1. If it's not, check that you have followed all the steps in [Software Setup Instructions](../../setup.md).*

## Querying for event information

The module `gwosc.datasets` provides tools for searching for datasets, including events, catalogs and full run strain data releases.


For example, we can search for events in the [GWTC-1 catalog](https://www.gw-openscience.org/eventapi/html/GWTC-1-confident/), the catalog of all events from the O1 and O2 observing runs. A list of available catalogs can be seen in the [Event Portal](https://gw-openscience.org/eventapi)

In [6]:
from gwosc.datasets import find_datasets
from gwosc import datasets

#-- List all available catalogs
print("List of available catalogs")
print(find_datasets(type="catalog"))

List of available catalogs
['GWTC', 'GWTC-1-confident', 'GWTC-1-marginal', 'GWTC-2', 'GWTC-2.1-auxiliary', 'GWTC-2.1-confident', 'GWTC-2.1-marginal', 'GWTC-3-confident', 'GWTC-3-marginal', 'IAS-O3a', 'Initial_LIGO_Virgo', 'O1_O2-Preliminary', 'O3_Discovery_Papers', 'O3_IMBH_marginal', 'O4_Discovery_Papers']


In [7]:
#-- Print all the GW events from the GWTC-1 catalog
gwtc1 = datasets.find_datasets(type='events', catalog='GWTC-1-confident')
print('GWTC-1 events:', gwtc1)
print("")

GWTC-1 events: ['GW150914-v3', 'GW151012-v3', 'GW151226-v2', 'GW170104-v2', 'GW170608-v3', 'GW170729-v1', 'GW170809-v1', 'GW170814-v3', 'GW170817-v3', 'GW170818-v1', 'GW170823-v1']



Note that the event name is of the type _GWyymmdd-vx_ where _x_ is the last available version for the data set provided by GWOSC.

In [8]:
#-- Print all the large strain data sets from LIGO/Virgo/KAGRA observing runs
runs = find_datasets(type='run')
print('Large data sets:', runs)

Large data sets: ['O1', 'O2', 'O3GK', 'O3a', 'O3b', 'S5', 'S6']


_Attention: Note that the most recent observation runs, e.g. O2, are labeled with names containing the name of the run (e.g. O2), the sampling rate (4 or 16 kHz) and the release version (e.g. R1). This means that for O2 you have two labels 'O2_4KHZ_R1' and 'O2_16KHZ_R1', depending which is the desired sampling rate_

`datasets.find_datasets` also accepts a `segment` and `detector` keyword to narrow results based on GPS time and detector:

In [9]:
#-- Detector and segments keywords limit search result
print(datasets.find_datasets(type='events', catalog='GWTC-1-confident', detector="L1", segment=(1164556817, 1187733618)))

['GW170104-v2', 'GW170608-v3', 'GW170729-v1', 'GW170809-v1', 'GW170814-v3', 'GW170817-v3', 'GW170818-v1', 'GW170823-v1']


Using `gwosc.datasets.event_gps`, we can query for the GPS time of a specific event (it works also without the version number):

In [10]:
from gwosc.datasets import event_gps
gps = event_gps('GW190425')
print(gps)

1240215503.0


<div class="alert alert-info">All of these times are returned in the GPS time system, which counts the number of seconds that have elapsed since the start of the GPS epoch at midnight (00:00) on January 6th 1980. GWOSC provides a <a href="https://www.gw-openscience.org/gps/">GPS time converter</a> you can use to translate into datetime, or you can use <a href="https://gwpy.github.io/docs/stable/time/"><code>gwpy.time</code></a>.</div>

In [11]:
# You can do also the vice-versa
from gwosc.datasets import event_at_gps
print(datasets.event_at_gps(1240215503))

GW190425


Note that the method `event_at_gps` looks for events found within 1 seconds of the given GPS time. If no events is found it will give an error.

We can query for the GPS time interval for an observing run:

In [15]:
from gwosc.datasets import run_segment
print(run_segment('O1'))

(1126051217, 1137254417)


In [13]:
# and vice-versa also in this case
from gwosc.datasets import run_at_gps
print(run_at_gps(1240215503))

O3a


Now we can use what we have learned with `run_segment` and `find_datasets` to see only the confident events in O1:

In [16]:
O1_events = datasets.find_datasets(type='events', catalog='GWTC-1-confident', segment=run_segment('O1'))
print(O1_events)

['GW150914-v3', 'GW151012-v3', 'GW151226-v2']


## Querying for data files

The `gwosc.locate` module provides a function to find the URLs of data files associated with a given dataset.

For event datasets, one can get the list of URLs using only the event name:

In [17]:
from gwosc.locate import get_event_urls
urls = get_event_urls('GW150914')
print(urls)

['https://gwosc.org/eventapi/json/GWTC-1-confident/GW150914/v3/H-H1_GWOSC_4KHZ_R1-1126259447-32.hdf5', 'https://gwosc.org/eventapi/json/GWTC-1-confident/GW150914/v3/H-H1_GWOSC_4KHZ_R1-1126257415-4096.hdf5', 'https://gwosc.org/eventapi/json/GWTC-1-confident/GW150914/v3/L-L1_GWOSC_4KHZ_R1-1126259447-32.hdf5', 'https://gwosc.org/eventapi/json/GWTC-1-confident/GW150914/v3/L-L1_GWOSC_4KHZ_R1-1126257415-4096.hdf5']


By default, this function returns all of the files associated with a given event, which isn't particularly helpful. However, we can filter on any of these by using keyword arguments, for example to get the URL for the 32-second file for the LIGO-Livingston detector:

In [18]:
urls = get_event_urls('GW150914', duration=32, detector='L1')
print(urls)

['https://gwosc.org/eventapi/json/GWTC-1-confident/GW150914/v3/L-L1_GWOSC_4KHZ_R1-1126259447-32.hdf5']


##  Query filtered by merger parameters
The `query_events` module of `gwosc.datasets` allows to get a list of events filtered by some parameters, similar to what is done by the `Query` function of the [event portal](https://www.gw-openscience.org/eventapi/html/query/). A list of available parameters can be found [here](https://gwosc.readthedocs.io/en/stable/reference/gwosc.datasets.query_events.html) or using `query_events?`.

Let's see how to use this module to find which events have been detected with a network signal to noise ratio (SNR) between 25 and 30:

In [19]:
from gwosc.datasets import query_events
selection = query_events(select=["25 <= network-matched-filter-snr <= 30"])
#this is equivalent to
#query_events(select=["network-matched-filter-snr <= 30", "network-matched-filter-snr>= 25"])
print(selection)

['GW200129_065458-v1', 'GW190814-v1', 'GW190814_211039-v3', 'GW190521_074359-v2', 'GW150914-v3']


Note that this module will give the list of **all available versions** for all the events that have the required parameters. For example, in this query the event GW190814 is listed twice because 2 versions of that event satisfy the request of SNR between 25 and 30.

# Exercises

Now that you've seen examples of how to query for dataset information using the `gwosc` package, please try and complete the following exercices using that interface:

- How many months did O2 last? (Hint: check the output of _find_datasets(type='run')_ to find the correct label to use)
- How many GWTC-3-confident events were detected during O3b?
- How many events have been detected with a network signal to noise ratio (SNR) >= 30?

- 1.-  Encontrando duracion de O2

In [24]:
duration_O2 = datasets.find_datasets(type='run')
#duration_O2

In [26]:
run_segment('O2')

(1164556817, 1187733618)

In [27]:
start_time_O2 = run_segment('O2')[0]
end_time_O2 = run_segment('O2')[1]

duration_in_seconds = end_time_O2 - start_time_O2

# Approximate conversion to years and then months
seconds_in_a_year = 365.25 * 24 * 60 * 60
duration_in_years = duration_in_seconds / seconds_in_a_year
duration_in_months = duration_in_years * 12

print(f"The duration of O2 was approximately {duration_in_months:.2f} months.")

The duration of O2 was approximately 8.81 months.


- 2.- Cuantos eventos confiables ocurrieron en GWTC-3 mientras ocurria O3b??

In [28]:
run_segment('O3b')

(1256655618, 1269363618)

In [29]:
O3b = datasets.find_datasets(type='events', catalog='GWTC-3-confident', segment=run_segment('O3b'))

In [30]:
O3b

['GW191103_012549-v1',
 'GW191105_143521-v1',
 'GW191109_010717-v1',
 'GW191113_071753-v1',
 'GW191126_115259-v1',
 'GW191127_050227-v1',
 'GW191129_134029-v1',
 'GW191204_110529-v1',
 'GW191204_171526-v1',
 'GW191215_223052-v1',
 'GW191216_213338-v1',
 'GW191219_163120-v1',
 'GW191222_033537-v1',
 'GW191230_180458-v1',
 'GW200112_155838-v1',
 'GW200115_042309-v2',
 'GW200128_022011-v1',
 'GW200129_065458-v1',
 'GW200202_154313-v1',
 'GW200208_130117-v1',
 'GW200208_222617-v1',
 'GW200209_085452-v1',
 'GW200210_092254-v1',
 'GW200216_220804-v1',
 'GW200219_094415-v1',
 'GW200220_061928-v1',
 'GW200220_124850-v1',
 'GW200224_222234-v1',
 'GW200225_060421-v1',
 'GW200302_015811-v1',
 'GW200306_093714-v1',
 'GW200308_173609-v1',
 'GW200311_115853-v1',
 'GW200316_215756-v1',
 'GW200322_091133-v1']

Que eventos fueron detectados con un SNR >= 30 (Network signal to noise radio)

In [31]:
eventos_filtrados = query_events(select=["network-matched-filter-snr >= 30"])
eventos_filtrados

['GW170817-v3']